In [5]:
import numpy as np
import pandas as pd
from library.constants import *
import math



SIMPLIFIED_JOINTS = ["Head", "Neck", "Left Upper Arm", "Right Upper Arm", "Left Forearm", "Right Forearm",
                     "Left Hand", "Right Hand", "Pelvis", "Left Upper Leg", "Right Upper Leg",
                     "Left Lower Leg", "Right Lower Leg", "Left Foot", "Right Foot"]

SKELETON_CONNECTION_MAP = [["Head", "Neck"],
                           ["Neck", "Left Upper Arm"],
                           ["Left Upper Arm", "Left Forearm"],
                           ["Left Forearm", "Left Hand"],
                           ["Neck", "Right Upper Arm"],
                           ["Right Upper Arm", "Right Forearm"],
                           ["Right Forearm", "Right Hand"],
                           ["Neck", "Pelvis"],
                           ["Pelvis", "Left Upper Leg"],
                           ["Left Upper Leg", "Left Lower Leg"],
                           ["Left Lower Leg", "Left Foot"],
                           ["Pelvis", "Right Upper Leg"],
                           ["Right Upper Leg", "Right Lower Leg"],
                           ["Right Lower Leg", "Right Foot"]]

used_cols = []
for joints in SIMPLIFIED_JOINTS:
    used_cols.append(joints + " x")
    used_cols.append(joints + " y")
    used_cols.append(joints + " z")

def read_data(file_name):
    return pd.read_excel(DATA_PATH + file_name,
                         sheet_name=RECORDING_TYPES,
                         usecols=used_cols)

DATA_PATH = "C:\\Users\\gaoch\\MA\\Badminton weights\\P02_B\\"
motion_recording = read_data("P02_B_noweightwithgloves.xlsx")


In [6]:
all_path = OUTPUT_FOLDER + "data.xlsx"
with pd.ExcelWriter(all_path) as writer:
    for recoding_type in RECORDING_TYPES:
        motion_recording[recoding_type][:].to_excel(writer, sheet_name=recoding_type, index=False)

In [7]:
print(motion_recording.keys())

dict_keys(['Segment Position', 'Segment Velocity', 'Segment Acceleration', 'Segment Angular Velocity', 'Segment Angular Acceleration'])
